# Automatic coastal vegetation edge detection

## Context
Coastal communities, land covers, and intertidal habitats are vulnerable receptors of erosion, flooding or both in combination. This vulnerability is likely to increase with sea level rise and greater storminess over future decadal-scale time periods [(Rogers et al., 2021)](https://www.tandfonline.com/doi/abs/10.1080/01431161.2021.1897185?journalCode=tres20).

`VEdge_Detector` is a python-based tool for the automated detection of coastal vegetation edges in remote sensing imagery. The tool produces a heatmap, showing the pixels predicted with the highest confidence as being the vegetation line (Figure 1).

<img src="https://github.com/MartinSJRogers/VEdge_Detector/blob/907ed83096570b87dcd211384c697d51097f3872/example_Images.png?raw=true" width="600">

**Figure 1** Outputs produced by the `VEdge_Detector` model in A) Suffolk, United Kingdom; B) Wilk auf Föhr, Germany and C) Varela, Guinea Bissau. Source: [Rogers (2021)](https://github.com/MartinSJRogers/VEdge_Detector).

The `VEdge_Detector` model was trained using Planet 3 – 5 m spatial resolution imagery. It has also detected vegetation edges in Landsat and Copernicus imagery, although performance is not guaranteed. The tool cannot detect the vegetation edge in aerial imagery.

<div class="alert alert-block alert-info">
In this notebook, we'll walk through how scivision can assist in discovering the `VEdge_detector` model available in the scivision branch of the model repository and then use it to generate predictions of coastal vegetation line over a sample satellite image. 
    
Further details of how the model has been trained and developed is outlined in the publication by [Rogers et al. (2021)](https://www.tandfonline.com/doi/abs/10.1080/01431161.2021.1897185?journalCode=tres20) .
</div>

## Load scivision

To get started, let's first load `scivision` modules relevant to this notebook.

In [13]:
from scivision.catalog import default_catalog
from scivision.io import load_pretrained_model, load_dataset

## Catalog

We'll use the VEdge_detector model which is registered in the scivision model catalog as `vegde-detector`. Then we'll look with `compatible_datasources` potential data entries registered in the scivision data catalog with tasks and format compatible with the model.

In [14]:
model_name = 'vegde-detector'

In [16]:
default_catalog.datasources.to_dataframe()

,name,description,tasks,domains,url,format,labels_provided,institution,tags
0,data-000,None,"(TaskEnum.object_detection, TaskEnum.segmentat...","(optical-microscopy,)",https://github.com/stardist/stardist/releases/...,image,True,"(epfl,)","(help-needed, 3D, cell, cell-counting, biology..."
1,data-001,None,"(TaskEnum.segmentation, TaskEnum.thresholding,...","(multispectral, hyperspectral)",https://gitlab.au.dk/AUENG-Vision/OPPD/-/archi...,image,True,"(Aarhus University,)","(help-needed, 2D, plant-phenotyping, agricultu..."
2,data-002,None,"(TaskEnum.classificiation,)","(oceanography,)",https://github.com/scotthosking/intake-plankto...,image,True,"(CEFAS,)","(help-needed, 2D)"
3,data-003,Koala,"(TaskEnum.object_detection,)","(computer-vision,)",https://github.com/alan-turing-institute/scivi...,image,True,"(Wikipedia by Erik Veland,)","(demo,)"
4,data-004,Sample dataset of Plankton required to demonst...,"(TaskEnum.classificiation,)","(computer-vision,)",https://raw.githubusercontent.com/alan-turing-...,image,True,"(Centre for Environment, Fisheries and Aquacul...","(2D, plankton, ecology, environmental-science)"
5,data-005,Sample dataset of satellite imagery required t...,"(TaskEnum.classificiation, TaskEnum.segmentation)","(computer-vision, earth-observation, ecology)",https://github.com/MartinSJRogers/VEdge_Detector,image,False,"(Cambridge University,)","(2D, satellite, remote-sensing, ecology, envir..."


In [4]:
compatible_datasources = default_catalog.compatible_datasources(model_name).to_dataframe()

This returns a `pandas.dataframe` with the number of potential datasets to pair the model with.

In [17]:
compatible_datasources

,name,description,tasks,domains,url,format,labels_provided,institution,tags
0,data-000,None,"(TaskEnum.object_detection, TaskEnum.segmentat...","(optical-microscopy,)",https://github.com/stardist/stardist/releases/...,image,True,"(epfl,)","(help-needed, 3D, cell, cell-counting, biology..."
1,data-001,None,"(TaskEnum.segmentation, TaskEnum.thresholding,...","(multispectral, hyperspectral)",https://gitlab.au.dk/AUENG-Vision/OPPD/-/archi...,image,True,"(Aarhus University,)","(help-needed, 2D, plant-phenotyping, agricultu..."
5,data-005,Sample dataset of satellite imagery required t...,"(TaskEnum.classificiation, TaskEnum.segmentation)","(computer-vision, earth-observation, ecology)",https://github.com/MartinSJRogers/VEdge_Detector,image,False,"(Cambridge University,)","(2D, satellite, remote-sensing, ecology, envir..."


## Model

The following lines will look for the model URL (in this case the GitHub repository) from the scivision model catalog. After, the `load_pretrained_model` function will recognise the URL and install dependencies if they are not yet in the current virtual environment.

In [18]:
models = default_catalog.models.to_dataframe()
targetmodel = models[models.name == model_name].url.item()
model = load_pretrained_model(targetmodel, branch='scivision', allow_install=True)

  Cloning https://github.com/MartinSJRogers/VEdge_Detector.git@scivision (to revision main) to /private/var/folders/l8/99_59fvn4bl2szm125grkgqw0000gr/T/pip-install-zn8isdpd/vedge-detector_2848cc13058f446582a6664e66fba8e9


  Running command git clone --filter=blob:none --quiet https://github.com/MartinSJRogers/VEdge_Detector.git@scivision /private/var/folders/l8/99_59fvn4bl2szm125grkgqw0000gr/T/pip-install-zn8isdpd/vedge-detector_2848cc13058f446582a6664e66fba8e9
  fatal: unable to access 'https://github.com/MartinSJRogers/VEdge_Detector.git@scivision/': The requested URL returned error: 400
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/MartinSJRogers/VEdge_Detector.git@scivision /private/var/folders/l8/99_59fvn4bl2szm125grkgqw0000gr/T/pip-install-zn8isdpd/vedge-detector_2848cc13058f446582a6664e66fba8e9 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/MartinSJRogers/VEdge_Detector.git@scivision /private/var/folders/l8/99_59fvn4bl2szm125grkgqw0000

CalledProcessError: Command '['/Users/acoca/anaconda3/envs/coastalvegedge-detection-scivision/bin/python', '-m', 'pip', 'install', 'git+https://github.com/MartinSJRogers/VEdge_Detector.git@scivision@main#egg=vedge_detector']' returned non-zero exit status 1.

In [ ]:
# let's explore the model object
model

## Load data

In [ ]:
cat = load_dataset('https://github.com/acocac/VEdge_Detector')

In [ ]:
dataset = cat.sample_image(sample_image='winterton').to_dask()

## Prediction and Visualisation

In [ ]:
pred = model.predict(X=dataset,provider='Planet')